In [1]:
# Import required libraries
import pandas as pd 
import numpy as np
from datetime import datetime as dt_obj
from datetime import timedelta

# Function to round number into multiple of 12 
# Used to round minutes in flare timestamp 
# This is done because the data in SHARP database has candance of 12 minutes 
def round(tm):
    discard = timedelta(minutes=tm.minute % 12,
                                 seconds=tm.second,
                                 microseconds=tm.microsecond)
    tm -= discard
    if discard >= timedelta(minutes=6):
        tm += timedelta(minutes=12)
    return tm

# Function to convert string into date_time object
def to_date(tstr):
    year   = int(tstr[:4])
    month  = int(tstr[5:7])
    day    = int(tstr[8:10])
    hour   = int(tstr[11:13])
    minute = int(tstr[14:16])
    return dt_obj(year, month, day, hour, minute)

# Function to convert two string into one date_time object
# Used to convert Max timestamp into date_time object
def to_date_max(tstr1, tstr2):
    year   = int(tstr1[:4])
    month  = int(tstr1[5:7])
    day    = int(tstr1[8:10])
    hour   = int(tstr2[:2])
    minute = int(tstr2[2:4])
    return dt_obj(year, month, day, hour, minute)

# Function to convert into flatten numpy array
def to_array(List):
    l = np.array(List)
    l = l.flatten()
    return l 

In [42]:
%%time
# Import the datasets

# Events dataset (NOAA)
XRA_FLA = pd.read_csv('XRA and FLA.csv')                # We interested in XRA and FLA events only
                                                        # XRA events are flare events detected in x-ray spectrum. Used as main basis to merge with SHARP dataset
                                                        # FLA events are flare events detected in optics spectrum. Used as complementary to determine undetected AR in SHARP 
# SHARP dataset
SHARP = pd.read_csv('dataset.csv')

# Drop the unwanted 'Unnamed: 0' column in events dataset (NOAA)
XRA_FLA.drop(columns = ['Unnamed: 0'])

Wall time: 10.6 s


,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars,Reg,Date
0,4260,1202,1209,1218,G14,5,XRA,1-8A,B1.9 1.4E-04,NaN,2010-01-01 12:09:00
1,4270,1233,1243,1300,G14,5,XRA,1-8A,B2.3 3.1E-04,NaN,2010-01-01 12:43:00
2,4280,2329,2333,2342,G14,5,XRA,1-8A,B1.1 8.9E-05,NaN,2010-01-01 23:33:00
3,4290 +,0310,0313,0319,////,5,XRA,1-8A,B1.1 4.4E-05,1039.0,2010-01-02 03:13:00
4,4300,0709,0724,0740,G14,5,XRA,1-8A,C1.0 1.2E-03,1039.0,2010-01-02 07:24:00
...,...,...,...,...,...,...,...,...,...,...,...
30495,9810 +,1619,1626,1631,G16,5,XRA,1-8A,B4.9 2.6E-04,NaN,2021-12-30 16:26:00
30496,9820 +,1903,1909,1914,G16,5,XRA,1-8A,B4.0 2.6E-04,NaN,2021-12-30 19:09:00
30497,9830 +,2018,2024,2034,G16,5,XRA,1-8A,B4.5 3.9E-04,NaN,2021-12-30 20:24:00
30498,9840 +,2234,2237,2241,G16,5,XRA,1-8A,B4.2 1.5E-04,NaN,2021-12-30 22:37:00


In [43]:
%%time
# Make a new column for the rounded time in events dataset (NOAA)
time = [round(to_date(i)) for i in XRA_FLA.Date]
XRA_FLA['rounded'] = time
XRA_FLA

Wall time: 138 ms


,Unnamed: 0,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars,Reg,Date,rounded
0,1,4260,1202,1209,1218,G14,5,XRA,1-8A,B1.9 1.4E-04,NaN,2010-01-01 12:09:00,2010-01-01 12:12:00
1,3,4270,1233,1243,1300,G14,5,XRA,1-8A,B2.3 3.1E-04,NaN,2010-01-01 12:43:00,2010-01-01 12:48:00
2,5,4280,2329,2333,2342,G14,5,XRA,1-8A,B1.1 8.9E-05,NaN,2010-01-01 23:33:00,2010-01-01 23:36:00
3,7,4290 +,0310,0313,0319,////,5,XRA,1-8A,B1.1 4.4E-05,1039.0,2010-01-02 03:13:00,2010-01-02 03:12:00
4,9,4300,0709,0724,0740,G14,5,XRA,1-8A,C1.0 1.2E-03,1039.0,2010-01-02 07:24:00,2010-01-02 07:24:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30495,118398,9810 +,1619,1626,1631,G16,5,XRA,1-8A,B4.9 2.6E-04,NaN,2021-12-30 16:26:00,2021-12-30 16:24:00
30496,118400,9820 +,1903,1909,1914,G16,5,XRA,1-8A,B4.0 2.6E-04,NaN,2021-12-30 19:09:00,2021-12-30 19:12:00
30497,118402,9830 +,2018,2024,2034,G16,5,XRA,1-8A,B4.5 3.9E-04,NaN,2021-12-30 20:24:00,2021-12-30 20:24:00
30498,118404,9840 +,2234,2237,2241,G16,5,XRA,1-8A,B4.2 1.5E-04,NaN,2021-12-30 22:37:00,2021-12-30 22:36:00


In [44]:
%%time
# Filtering only for XRA events
XRA = XRA_FLA.loc[XRA_FLA['Type'] == 'XRA']
NOAA_Numbered = XRA.loc[XRA['Reg'] > 0]                 # Filter events with valid NOAA ARNUM (Reg) only

# Create a new column for NOAA ARNUM (Reg to ARNUM = Reg + 10000)
ARNUM = [int(i) + 10000 for i in NOAA_Numbered.Reg]                                                  
NOAA_Numbered['AR'] = ARNUM

# Import conversion tables for between NOAA ARNUM and HARPNUM 
conversion = pd.read_csv('conversion.csv')

Wall time: 21 ms


<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
%%time 
# Merge the NOAA_Numbered events with conversion dataset to obtain events dataset with HARPNUM
merged_XRA = pd.merge(NOAA_Numbered, conversion, on = ['AR'], how = 'left')

# Rename the columns in order to match SHARP dataset column names 
merged_XRA = merged_XRA.rename(columns = {"AR": "NOAA_AR", "HARP": "HARPNUM", 'rounded': 'date_time'})

# Convert Reg from float into int
merged_XRA['Reg'] = [int(i) for i in merged_XRA['Reg']]

# Create a new column for CLASS (flare class with number)
merged_XRA['CLASS'] = [i[0:2] for i in merged_XRA['Particulars']]

# Create a new column for class (flare class alphabet only)
merged_XRA['class'] = [i[0:1] for i in merged_XRA['Particulars']]
merged_XRA

Wall time: 23 ms


,Unnamed: 0,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars,Reg,Date,date_time,NOAA_AR,HARPNUM,CLASS,class
0,7,4290 +,0310,0313,0319,////,5,XRA,1-8A,B1.1 4.4E-05,1039,2010-01-02 03:13:00,2010-01-02 03:12:00,11039,NaN,B1,B
1,9,4300,0709,0724,0740,G14,5,XRA,1-8A,C1.0 1.2E-03,1039,2010-01-02 07:24:00,2010-01-02 07:24:00,11039,NaN,C1,C
2,12,4310,0758,0805,0813,G14,5,XRA,1-8A,B6.4 4.3E-04,1039,2010-01-02 08:05:00,2010-01-02 08:00:00,11039,NaN,B6,B
3,17,4330,0901,0910,0923,G14,5,XRA,1-8A,B4.5 4.8E-04,1039,2010-01-02 09:10:00,2010-01-02 09:12:00,11039,NaN,B4,B
4,30,4390,2239,2246,2255,G14,5,XRA,1-8A,B4.3 2.8E-04,1039,2010-01-02 22:46:00,2010-01-02 22:48:00,11039,NaN,B4,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14643,118343,9580 +,1232,1240,1257,G16,5,XRA,1-8A,B5.1 3.1E-04,2916,2021-12-29 12:40:00,2021-12-29 12:36:00,12916,NaN,B5,B
14644,118345,9590 +,2143,2149,2154,G16,5,XRA,1-8A,B5.5 3.1E-04,2921,2021-12-29 21:49:00,2021-12-29 21:48:00,12921,NaN,B5,B
14645,118347,9600 +,2312,2328,2340,G16,5,XRA,1-8A,B6.8 9.6E-04,2918,2021-12-29 23:28:00,2021-12-29 23:24:00,12918,NaN,B6,B
14646,118350,9610 +,0015,0024,0031,G16,5,XRA,1-8A,B7.7 6.4E-04,2921,2021-12-30 00:15:00,2021-12-30 00:12:00,12921,NaN,B7,B


In [46]:
# Convert date in SHARP dataset into date_time object 
date_format_str = '%Y.%m.%d_%H:%M:%S_TAI'  
SHARP['date_time'] = [dt_obj.strptime(i, date_format_str) for i in SHARP.T_REC]
SHARP

,Unnamed: 0,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,...,SHRGT45,MEANSHR,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,MEANJZD,MEANALP,date_time
0,0,2010.05.02_00:00:00_TAI,1,11067,11067,-65.205566,24.177683,298.638,1.878875e+22,5.949744e+12,...,1.832,17.819,23.882,72.810,75.884,31.044,-0.001544,-0.083294,-0.005115,2010-05-02 00:00:00
1,1,2010.05.02_00:12:00_TAI,1,11067,11067,-64.984444,24.106541,301.313,1.905348e+22,5.977893e+12,...,2.157,18.166,24.393,71.647,74.411,29.846,-0.002460,-0.176605,-0.008261,2010-05-02 00:12:00
2,2,2010.05.02_00:24:00_TAI,1,11067,11067,-64.888580,24.135088,312.556,1.954166e+22,6.152751e+12,...,2.322,18.145,24.500,72.839,75.320,30.416,-0.002394,-0.197845,-0.007963,2010-05-02 00:24:00
3,3,2010.05.02_00:36:00_TAI,1,11067,11067,-64.758209,24.144226,316.364,2.166810e+22,6.234756e+12,...,2.927,18.561,24.714,73.475,76.219,31.189,-0.002040,-0.162419,-0.006730,2010-05-02 00:36:00
4,4,2010.05.02_00:48:00_TAI,1,11067,11067,-64.534355,24.097631,331.849,2.409164e+22,7.389016e+12,...,4.633,19.909,25.323,75.065,81.613,32.784,-0.001154,-0.143143,-0.003972,2010-05-02 00:48:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900684,2900684,2021.12.30_23:12:00_TAI,7912,12923,12923,37.216583,-28.806856,17.246,1.251446e+21,3.618160e+11,...,11.260,27.357,38.822,148.169,149.718,66.568,0.002266,0.177169,0.006263,2021-12-30 23:12:00
2900685,2900685,2021.12.30_23:24:00_TAI,7912,12923,12923,37.328621,-28.814804,19.899,1.381736e+21,4.292386e+11,...,12.528,28.594,41.130,134.921,139.982,62.222,-0.000042,1.449496,-0.000129,2021-12-30 23:24:00
2900686,2900686,2021.12.30_23:36:00_TAI,7912,12923,12923,37.393547,-28.820169,23.266,1.554695e+21,5.211204e+11,...,20.154,32.024,43.085,122.141,128.427,52.895,-0.002677,0.852646,-0.009535,2021-12-30 23:36:00
2900687,2900687,2021.12.30_23:48:00_TAI,7912,12923,12923,37.471878,-28.819061,24.068,1.533791e+21,5.515389e+11,...,14.206,29.845,40.497,126.887,131.375,53.077,-0.002370,0.416960,-0.008219,2021-12-30 23:48:00


In [52]:
%%time
# SHARP dataset isn't all complete. There are datapoints which do not have HARPUM and/or NOAA ARNUM
# Hence we try to merge the dataset from two parameters: NOAA ARNUM and HARPNUM
# This is in order to detect more flare datapoints in SHARP dataset

# First we need to make a dataset with a few columns dropped in order to obtain simpler merged dataset 
# Merging from NOAA ARNUM
for_merging_NOAA = merged_XRA.drop(columns = ['Event', 'Begin', 'End', 'Obs', 'Q', 'Loc/Frq', 'Date', 'HARPNUM', 'Particulars'])
# Merging from HARPNUM
for_merging_HARPNUM = merged_XRA.drop(columns = ['Event', 'Begin', 'End', 'Obs', 'Q', 'Loc/Frq', 'Date', 'NOAA_AR', 'Particulars'])

# Next, do the merging
merged_on_NOAA = pd.merge(SHARP, for_merging_NOAA, on=['NOAA_AR', 'date_time'], how="left")
merged_on_HARPNUM = pd.merge(SHARP, for_merging_HARPNUM, on=['HARPNUM', 'date_time'], how="left")


# Combine the two merged dataset into one large dataset
events_in_SHARP = pd.concat([merged_on_NOAA.loc[merged_on_NOAA['Reg'] >= 0], merged_on_HARPNUM.loc[merged_on_HARPNUM['Reg'] >= 0]])
# Drop the NaN in 'Reg' column 
events_in_SHARP = events_in_SHARP.dropna(subset = ['Reg'])
events_in_SHARP

Wall time: 4.34 s


,Unnamed: 0_x,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,...,MEANJZH,MEANJZD,MEANALP,date_time,Unnamed: 0_y,Max,Type,Reg,CLASS,class
31,31,2010.05.02_06:12:00_TAI,1,11067,11067,-61.366440,23.991444,338.064,2.308178e+22,6.776103e+12,...,-0.001447,-0.312196,-0.004762,2010-05-02 06:12:00,2058.0,0609,XRA,1067.0,B2,B
51,51,2010.05.02_10:12:00_TAI,1,11067,11067,-58.990116,23.704386,440.462,3.160769e+22,8.957106e+12,...,0.000530,-0.263572,0.001691,2010-05-02 10:12:00,2060.0,1009,XRA,1067.0,B3,B
1125,1125,2010.05.03_21:48:00_TAI,10,11066,11066,-5.943328,-26.321066,152.483,1.095656e+22,2.751868e+12,...,-0.019968,0.764016,-0.044415,2010-05-03 21:48:00,2081.0,2153,XRA,1066.0,B1,B
1179,1179,2010.05.03_13:00:00_TAI,11,11063,11063,47.102375,17.676003,99.427,1.023441e+22,1.769452e+12,...,0.013131,0.528098,0.026093,2010-05-03 13:00:00,2074.0,1257,XRA,1063.0,B2,B
1687,1687,2010.05.04_00:24:00_TAI,8,11069,11069,15.619510,41.304546,51.137,3.305052e+21,1.308026e+12,...,-0.005654,-1.133347,-0.023305,2010-05-04 00:24:00,2104.0,0023,XRA,1069.0,B1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727507,2727472,2020.12.30_04:24:00_TAI,7513,12795,12795,12.648386,-18.474335,441.213,6.568917e+22,8.721554e+12,...,-0.003850,0.458816,-0.008854,2020-12-30 04:24:00,110251.0,0421,XRA,2795.0,B1,B
2727523,2727488,2020.12.30_07:36:00_TAI,7513,12795,12795,14.446938,-18.433222,462.510,5.833678e+22,9.297702e+12,...,-0.002455,0.843509,-0.006208,2020-12-30 07:36:00,110257.0,0735,XRA,2795.0,B1,B
2728269,2728234,2021.01.01_02:00:00_TAI,7513,12795,12795,37.738396,-18.779419,374.843,3.094606e+22,7.962530e+12,...,-0.003554,0.311293,-0.011755,2021-01-01 02:00:00,110288.0,0154,XRA,2795.0,B3,B
2728283,2728248,2021.01.01_04:48:00_TAI,7513,12795,12795,39.406322,-18.828533,401.450,3.137842e+22,8.673221e+12,...,-0.004743,-0.048957,-0.014985,2021-01-01 04:48:00,110290.0,0442,XRA,2795.0,B2,B


In [54]:
# The dataset 'events_in_dataset' contains duplicate flare events
# We remove duplicate datapoint of flare events by looking at value in 'Unnamed: 0_x' column
events_in_SHARP = events_in_SHARP.drop_duplicates(subset = ['Unnamed: 0_x'])

# Drop the unwanted 'Unnamed: 0' column 
events_in_SHARP = events_in_SHARP.drop(columns = ['Unnamed: 0_x'])
# Drop the duplicate datapoints by looking at every value in datapoints 
events_in_SHARP = events_in_SHARP.drop_duplicates()
events_in_SHARP

,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,MEANJZH,MEANJZD,MEANALP,date_time,Unnamed: 0_y,Max,Type,Reg,CLASS,class
31,2010.05.02_06:12:00_TAI,1,11067,11067,-61.366440,23.991444,338.064,2.308178e+22,6.776103e+12,17.691,...,-0.001447,-0.312196,-0.004762,2010-05-02 06:12:00,2058.0,0609,XRA,1067.0,B2,B
51,2010.05.02_10:12:00_TAI,1,11067,11067,-58.990116,23.704386,440.462,3.160769e+22,8.957106e+12,7.804,...,0.000530,-0.263572,0.001691,2010-05-02 10:12:00,2060.0,1009,XRA,1067.0,B3,B
1125,2010.05.03_21:48:00_TAI,10,11066,11066,-5.943328,-26.321066,152.483,1.095656e+22,2.751868e+12,42.412,...,-0.019968,0.764016,-0.044415,2010-05-03 21:48:00,2081.0,2153,XRA,1066.0,B1,B
1179,2010.05.03_13:00:00_TAI,11,11063,11063,47.102375,17.676003,99.427,1.023441e+22,1.769452e+12,24.214,...,0.013131,0.528098,0.026093,2010-05-03 13:00:00,2074.0,1257,XRA,1063.0,B2,B
1687,2010.05.04_00:24:00_TAI,8,11069,11069,15.619510,41.304546,51.137,3.305052e+21,1.308026e+12,7.344,...,-0.005654,-1.133347,-0.023305,2010-05-04 00:24:00,2104.0,0023,XRA,1069.0,B1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709984,2020.11.23_23:00:00_TAI,7477,"12785,12786,12788",12786,-70.260025,-18.571222,1539.699,1.140349e+24,2.570838e+13,281.404,...,0.005910,-0.293075,0.009407,2020-11-23 23:00:00,109415.0,2255,XRA,2785.0,C1,C
2710258,2020.11.24_05:12:00_TAI,7477,"12785,12786,12788",12786,-67.075958,-18.563137,1709.853,1.157601e+24,3.033773e+13,269.382,...,0.004618,-0.305524,0.008193,2020-11-24 05:12:00,109425.0,0517,XRA,2785.0,C1,C
2710261,2020.11.24_05:48:00_TAI,7477,"12785,12786,12788",12786,-66.742470,-18.581125,1795.231,1.164359e+24,3.202051e+13,249.698,...,0.004039,-0.273760,0.007406,2020-11-24 05:48:00,109427.0,0542,XRA,2785.0,C1,C
2710603,2020.11.25_00:24:00_TAI,7477,"12785,12786,12788",12786,-57.816357,-18.569885,2880.677,1.297173e+24,5.365559e+13,238.827,...,0.002699,-0.200461,0.005248,2020-11-25 00:24:00,109446.0,0025,XRA,2785.0,B7,B


From this merging operation, we only detect 12514 flare events in SHARP database out of 14648 flares from NOAA XRA dataset. 63 of them are M or X class.

In [58]:
lost = pd.read_csv('lost flares.csv', sep=';')
lost.drop(columns = ['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])

,CLASS,NOAA_AR,HARPNUM,date_time,Max
0,X1,11748,2748.0,2013-05-13 02:12:00,217
1,X1,12158,0.0,2014-09-10 17:48:00,1745
2,X8,12673,7115.0,2017-09-10 16:12:00,1606
3,M1,11149,345.0,2011-01-28 00:48:00,103
4,M1,11302,892.0,2011-09-21 12:24:00,1223
...,...,...,...,...,...
58,M2,12891,0.0,2021-11-09 17:00:00,1702
59,M1,12898,0.0,2021-12-05 07:24:00,719
60,M1,12908,0.0,2021-12-20 11:36:00,1136
61,M1,12916,0.0,2021-12-21 07:48:00,750


We can possibly search these missing flares by looking at AR time and locations in SHARP dataset and matching it with FLA events in NOAA FLA dataset. But from this arduous searching, we only managed to find 2 more flares which are: 
- HARPNUM 4530 at 2014.09.14_02:12:00_TAI
- HARPNUM 4613 AT 2014.09.28_17:36:00_TAI

In [55]:
# Collecting the missing flare datapoints
ADD1 = SHARP.loc[SHARP['date_time'] == '2014-09-14 02:12:00'].iloc[0].to_frame().transpose()
ADD2 = SHARP.loc[SHARP['date_time'] == '2014-09-28 17:36:00'].iloc[5].to_frame().transpose()

# Combining the missing flare datapoints and adding it to events_in_SHARP dataset 
ADD = pd.concat([ADD1, ADD2], ignore_index=True)
ADD['Max'] = ['0212', '1734']
ADD['Type'] = ['FLA', 'FLA']
ADD['Reg'] = ['2157', '2173']
ADD['CLASS'] = ['M1', 'M1']
ADD['class'] = ['M', 'M']

events_in_SHARP = pd.concat([events_in_SHARP, ADD], ignore_index=True)
events_in_SHARP

,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,MEANJZD,MEANALP,date_time,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0
0,2010.05.02_06:12:00_TAI,1,11067,11067,-61.36644,23.991444,338.064,23081779999999998820352.0,6776103000000.0,17.691,...,-0.312196,-0.004762,2010-05-02 06:12:00,2058.0,0609,XRA,1067.0,B2,B,NaN
1,2010.05.02_10:12:00_TAI,1,11067,11067,-58.990116,23.704386,440.462,31607689999999999934464.0,8957106000000.0,7.804,...,-0.263572,0.001691,2010-05-02 10:12:00,2060.0,1009,XRA,1067.0,B3,B,NaN
2,2010.05.03_21:48:00_TAI,10,11066,11066,-5.943328,-26.321066,152.483,10956559999999999475712.0,2751868000000.0,42.412,...,0.764016,-0.044415,2010-05-03 21:48:00,2081.0,2153,XRA,1066.0,B1,B,NaN
3,2010.05.03_13:00:00_TAI,11,11063,11063,47.102375,17.676003,99.427,10234410000000000983040.0,1769452000000.0,24.214,...,0.528098,0.026093,2010-05-03 13:00:00,2074.0,1257,XRA,1063.0,B2,B,NaN
4,2010.05.04_00:24:00_TAI,8,11069,11069,15.61951,41.304546,51.137,3305052000000000131072.0,1308026000000.0,7.344,...,-1.133347,-0.023305,2010-05-04 00:24:00,2104.0,0023,XRA,1069.0,B1,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12511,2020.11.24_05:48:00_TAI,7477,"12785,12786,12788",12786,-66.74247,-18.581125,1795.231,1164359000000000054788096.0,32020510000000.0,249.698,...,-0.27376,0.007406,2020-11-24 05:48:00,109427.0,0542,XRA,2785.0,C1,C,NaN
12512,2020.11.25_00:24:00_TAI,7477,"12785,12786,12788",12786,-57.816357,-18.569885,2880.677,1297172999999999879151616.0,53655590000000.0,238.827,...,-0.200461,0.005248,2020-11-25 00:24:00,109446.0,0025,XRA,2785.0,B7,B,NaN
12513,2020.12.04_19:24:00_TAI,7477,"12785,12786,12788",12786,64.155121,-17.595686,1742.816,592874199999999991873536.0,34636820000000.0,18.71,...,-0.105532,-0.00076,2020-12-04 19:24:00,109905.0,1925,XRA,2788.0,B6,B,NaN
12514,2014.09.14_02:12:00_TAI,4530,MISSING,0,48.16206,-14.862936,2139.137,257392400000000001572864.0,48533460000000.0,239.125,...,-0.100753,0.011405,2014-09-14 02:12:00,NaN,0212,FLA,2157,M1,M,1645389


After we have all flare events in SHARP dataset, we need to clean it. We found two invalid value for Max in index 747 and 11318. 

In [60]:
print(events_in_SHARP.Max.iloc[747])
print(events_in_SHARP.Max.iloc[11318])

U0908
U0123


In [61]:
# Change the invalid value into its proper value
events_in_SHARP.Max.iloc[747] = '0908'
events_in_SHARP.Max.iloc[11318] = '0123'

C:\Users\joko\AppData\Local\Temp\ipykernel_14532\3153144931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_in_SHARP.Max.iloc[747] = '0908'
C:\Users\joko\AppData\Local\Temp\ipykernel_14532\3153144931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_in_SHARP.Max.iloc[11318] = '0123'


There are other invalid value in Max which are '////' . We also need to replace them. 

In [63]:
events_in_SHARP.loc[events_in_SHARP['Max'] == '////']

,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,MEANJZD,MEANALP,date_time,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0
939,2011.04.11_00:00:00_TAI,495,"11190,11192",11190,-38.949055,13.282899,628.291,40625010000000001769472.0,14459700000000.0,59.904,...,-0.013699,0.012373,2011-04-11 00:00:00,9201.0,////,XRA,1190.0,C2,C,NaN
2054,2012.04.05_00:00:00_TAI,1528,11450,11450,18.541235,15.21335,952.901,180910100000000008257536.0,21392560000000.0,42.094,...,0.254069,0.005534,2012-04-05 00:00:00,22997.0,////,XRA,1450.0,C1,C,NaN
5237,2014.05.25_00:00:00_TAI,4138,"12065,12068,12072,12074",12065,52.420055,-17.383696,2854.173,398377800000000011272192.0,51774360000000.0,239.028,...,-0.089901,0.010022,2014-05-25 00:00:00,60658.0,////,XRA,2065.0,C3,C,NaN
8510,2017.03.26_00:12:00_TAI,6972,12644,12644,-45.821579,12.481902,412.28,34341179999999999606784.0,8240536000000.0,40.345,...,0.190546,-0.013616,2017-03-26 00:12:00,98613.0,////,XRA,2644.0,C1,C,NaN
10039,2021.04.17_00:00:00_TAI,7569,12814,12814,43.115944,-22.565096,384.903,30991350000000000065536.0,8520881000000.0,11.419,...,0.253362,-0.003745,2021-04-17 00:00:00,110922.0,////,XRA,2814.0,B4,B,NaN


As we can see, these data points aren't aren't coresponding to X or M class flare. Hence, we can replace the Max with '0000'  

In [67]:
%%time
# Create a blank list for storing Max value
MAX = []
# Change the Max value if invalid ('////')
# Else keep the Max value
# Store the Max value as date_time object
for i in range(len(events_in_SHARP)):
    if events_in_SHARP.Max[i] == '////':
        events_in_SHARP.Max[i] = '0000'
    m = to_date_max(events_in_SHARP.T_REC[i], events_in_SHARP.Max[i])
    MAX.append(m)

# Store the Max which contain date_time objects of Max timestamp into events_in_SHARP dataset
events_in_SHARP['Max'] = MAX

Wall time: 369 ms


For convinience, let's create two more columns: 'flare_index' & 'X_or_M'. 'flare_index' denotes the chronological sequence number of X or M class flares. 'X_or_M' shows if the datapoint contains X or M class flare. These columns will be useful later. 

In [71]:
%%time
# Get a variable which contain 'class' (alphabet) column
Class = to_array(events_in_SHARP['class'])
# Create blank list for the wanted columns
flare_index = []
X_or_M = []
# Make a dummy variable for 'flare_index'
count = 1 

# Assign value 'Y' if datapoint contain X or M class flare, else: 'N'
# Assign value = count if datapoint contain X or M class flare, else: 0
for i in range(len(MAX)):
    if (Class[i] == 'X') or (Class[i] == 'M'):
        X_or_M.append('Y')
        flare_index.append(count)
        count += 1
    else: 
        X_or_M.append('N')
        flare_index.append('0')

events_in_SHARP['X_or_M'] = X_or_M
events_in_SHARP['flare_index'] = flare_index
events_in_SHARP

Wall time: 9.04 ms


,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,date_time,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0,X_or_M,flare_index
0,2010.05.02_06:12:00_TAI,1,11067,11067,-61.36644,23.991444,338.064,23081779999999998820352.0,6776103000000.0,17.691,...,2010-05-02 06:12:00,2058.0,2010-05-02 06:09:00,XRA,1067.0,B2,B,NaN,N,0
1,2010.05.02_10:12:00_TAI,1,11067,11067,-58.990116,23.704386,440.462,31607689999999999934464.0,8957106000000.0,7.804,...,2010-05-02 10:12:00,2060.0,2010-05-02 10:09:00,XRA,1067.0,B3,B,NaN,N,0
2,2010.05.03_21:48:00_TAI,10,11066,11066,-5.943328,-26.321066,152.483,10956559999999999475712.0,2751868000000.0,42.412,...,2010-05-03 21:48:00,2081.0,2010-05-03 21:53:00,XRA,1066.0,B1,B,NaN,N,0
3,2010.05.03_13:00:00_TAI,11,11063,11063,47.102375,17.676003,99.427,10234410000000000983040.0,1769452000000.0,24.214,...,2010-05-03 13:00:00,2074.0,2010-05-03 12:57:00,XRA,1063.0,B2,B,NaN,N,0
4,2010.05.04_00:24:00_TAI,8,11069,11069,15.61951,41.304546,51.137,3305052000000000131072.0,1308026000000.0,7.344,...,2010-05-04 00:24:00,2104.0,2010-05-04 00:23:00,XRA,1069.0,B1,B,NaN,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12511,2020.11.24_05:48:00_TAI,7477,"12785,12786,12788",12786,-66.74247,-18.581125,1795.231,1164359000000000054788096.0,32020510000000.0,249.698,...,2020-11-24 05:48:00,109427.0,2020-11-24 05:42:00,XRA,2785.0,C1,C,NaN,N,0
12512,2020.11.25_00:24:00_TAI,7477,"12785,12786,12788",12786,-57.816357,-18.569885,2880.677,1297172999999999879151616.0,53655590000000.0,238.827,...,2020-11-25 00:24:00,109446.0,2020-11-25 00:25:00,XRA,2785.0,B7,B,NaN,N,0
12513,2020.12.04_19:24:00_TAI,7477,"12785,12786,12788",12786,64.155121,-17.595686,1742.816,592874199999999991873536.0,34636820000000.0,18.71,...,2020-12-04 19:24:00,109905.0,2020-12-04 19:25:00,XRA,2788.0,B6,B,NaN,N,0
12514,2014.09.14_02:12:00_TAI,4530,MISSING,0,48.16206,-14.862936,2139.137,257392400000000001572864.0,48533460000000.0,239.125,...,2014-09-14 02:12:00,NaN,2014-09-14 02:12:00,FLA,2157,M1,M,1645389,Y,721


Let's make one more column for events_in_SHARP dataset. The column will denotes if the flare in datapoint is an independent flare. Independent flare defined as flare which occurs after dt time from previous flare in the same AR. We called this column by the name 'LOOK'. 

In [72]:
%%time
# First, get datapoints which contain X or M class flare from events_in_SHARP dataset  
XnM = events_in_SHARP.loc[(events_in_SHARP['class'] == 'M') | (events_in_SHARP['class'] == 'X')]
# Get its HARPNUM list and drop the duplicates
HARPNUM = to_array(XnM['HARPNUM'].drop_duplicates())
# Then, get HARPNUM list of every flares and drop the duplicates
HARP = to_array(events_in_SHARP['HARPNUM'].drop_duplicates())
# Make a blank list for 'LOOK' column
LOOK = []
# dt = hour
hour = 24

# We then do a loop search for every datapoint in events_in_SHARP dataset
for i in range(len(events_in_SHARP)):
    # Store the event (flare) which is being investigated as event
    event = events_in_SHARP.iloc[i]
    # Get the date_time for the frist observation of the AR which corespond with the event
    # We need this information in order to determine if the event is the first flare detected from this AR
    zero = SHARP.loc[SHARP['HARPNUM'] == event['HARPNUM']].date_time.iloc[0]
    # Get the flares which corespond with said AR (HARPNUM) and assign index for each flare
    flares = events_in_SHARP.loc[events_in_SHARP['HARPNUM'] == event['HARPNUM']].reset_index()
    # Get the index of the flare which corespond with the event (the flare which is being investigated)
    index_of_event = flares.date_time[flares.date_time == event['date_time']].index[0]
    # Store the timestamp of each flare
    timestamp = to_array(flares['date_time'])
    # We then assign the variable for 'LOOK' column
    # The conditions are
    # Only look for X or M class flare
    # If it is the first of the AR and have temporal distance >= dt from the first observation of AR, assign it 'Y'
    # Elif it is not the first but have temporal distance >= dt from previous flare, assign it 'Y' 
    # Else assign it 'N'
    if event['X_or_M'] == 'Y': # If X or M class flare
        if index_of_event != 0: # If not the first class flare
            if timestamp[index_of_event] - timestamp[index_of_event - 1] >= hour*3600*10**9: # If the temporal distance >= dt
                LOOK.append('Y')
            else:  # If the temporal distance < dt 
                LOOK.append('N')
        else: # If is the first flare
            if (timestamp[index_of_event] - zero) >= timedelta(hours = hour):  # If the temporal distance >= dt
                LOOK.append('Y')
            else: # If the temporal distance < dt
                LOOK.append('N')
    else: # If not X or M class flare
        LOOK.append('N')

events_in_SHARP['LOOK'] = LOOK
events_in_SHARP.loc[events_in_SHARP['LOOK'] == 'Y']

Wall time: 1min 8s


,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0,X_or_M,flare_index,LOOK
156,2010.08.07_18:24:00_TAI,115,"11093,11099",11093,-33.313995,12.95864,806.86,225115200000000006291456.0,17044910000000.0,152.767,...,3273.0,2010-08-07 18:24:00,XRA,1093.0,M1,M,NaN,Y,4,Y
1452,2011.09.06_01:48:00_TAI,833,11283,11283,3.095427,14.939167,1703.639,357961500000000013238272.0,34651480000000.0,157.447,...,13954.0,2011-09-06 01:50:00,XRA,1283.0,M5,M,NaN,Y,54,Y
1637,2011.10.22_11:12:00_TAI,940,"11314,11319",11314,84.104347,17.975479,636.026,105921899999999995936768.0,13016090000000.0,25.05,...,17149.0,2011-10-22 11:10:00,XRA,1314.0,M1,M,NaN,Y,87,Y
1722,2011.11.15_12:48:00_TAI,1066,"11346,11351",11346,-34.526257,-20.454929,1374.598,195770399999999986368512.0,29035060000000.0,128.001,...,18258.0,2011-11-15 12:43:00,XRA,1346.0,M1,M,NaN,Y,98,Y
1873,2012.01.23_04:00:00_TAI,1321,"11401,11402,11405,11407",11402,24.041374,22.252535,5198.877,865671400000000001835008.0,104253400000000.0,1327.557,...,21208.0,2012-01-23 03:59:00,XRA,1402.0,M8,M,NaN,Y,100,Y
2003,2012.03.13_17:36:00_TAI,1449,"11429,11430",11429,60.047962,18.068813,2739.847,830472399999999978504192.0,49495350000000.0,972.815,...,22411.0,2012-03-13 17:41:00,XRA,1429.0,M7,M,NaN,Y,118,Y
2005,2012.03.14_15:24:00_TAI,1461,11432,11432,-6.292967,14.49999,827.725,73035200000000002097152.0,18719350000000.0,32.027,...,22443.0,2012-03-14 15:21:00,XRA,1432.0,M2,M,NaN,Y,119,Y
2014,2012.03.17_20:36:00_TAI,1464,11434,11434,21.733738,-22.19223,452.718,43442430000000000851968.0,8981308000000.0,112.302,...,22520.0,2012-03-17 20:39:00,XRA,1434.0,M1,M,NaN,Y,120,Y
2308,2012.06.09_11:36:00_TAI,1750,"11504,11505",11504,-74.569725,-15.513855,872.398,148390099999999993577472.0,13872560000000.0,182.395,...,26333.0,2012-06-09 11:32:00,XRA,1504.0,M1,M,NaN,Y,136,Y
2519,2012.07.28_21:00:00_TAI,1879,"11529,11530,11532,11533,11536",11532,-28.382008,-18.481388,4457.487,957865300000000039714816.0,86327660000000.0,173.79,...,30117.0,2012-07-28 20:56:00,XRA,1532.0,M6,M,NaN,Y,177,Y


If you want to investigate the corelation between dt and the number of 'Y' in 'LOOK' column, run the following code (long execution). 

In [ ]:
"""
%%time
x_axis = np.arange(1, 73)
y_axis = []
for z in range(1, 73):
    XnM = events_in_dataset.loc[(events_in_dataset['class'] == 'M') | (events_in_dataset['class'] == 'X')]
    HARPNUM = to_array(XnM['HARPNUM'].drop_duplicates())
    HARP = to_array(events_in_dataset['HARPNUM'].drop_duplicates())
    LOOK = []
    hour = z

    for i in range(len(events_in_dataset)):
        event = events_in_dataset.iloc[i]
        zero = DF.loc[DF['HARPNUM'] == event['HARPNUM']].date_time.iloc[0]
        flares = events_in_dataset.loc[events_in_dataset['HARPNUM'] == event['HARPNUM']].reset_index()
        index_of_event = flares.date_time[flares.date_time == event['date_time']].index[0]
        timestamp = to_array(flares['date_time'])
        if event['X_or_M'] == 'Y': #jika flare kelas X atau M
            if index_of_event != 0: #jika bukan flare pertama
                if timestamp[index_of_event] - timestamp[index_of_event - 1] >= hour*3600*10**9: #jika jarak flare dari flare sebelum lebih dari <hour> jam
                    LOOK.append('Y')
                else:  #jika jarak flare dari awal kurang dari <hour> jam 
                    LOOK.append('N')
            else: #jika flare pertama
                if (timestamp[index_of_event] - zero) >= timedelta(hours = hour):
                    LOOK.append('Y')
                else: #jika jarak flare dari awal kurang dari <hour> jam 
                    LOOK.append('N')
        else: #jika flare bukan kelas X atau M
            LOOK.append('N')

    events_in_dataset['LOOK'] = LOOK
    y_axis.append(len(events_in_dataset.loc[events_in_dataset['LOOK'] == 'Y']))

plt.plot(x_axis, y_axis, 'o')
plt.show()
"""

As for now, we have completely obtain all the information of flare events which could be detected in SHARP dataset. All we need to do now is to merge this dataset with the SHARP dataset. 

In [75]:
%%time
# Merge the dataset to obtain complete dataset
complete_dataset = pd.merge(SHARP, events_in_SHARP, on=['T_REC', 'HARPNUM', 'NOAA_ARS', 'NOAA_AR', 'LON_FWT', 'LAT_FWT',
       'TOTUSJH', 'TOTPOT', 'TOTUSJZ', 'ABSNJZH', 'SAVNCPP', 'USFLUX',
       'AREA_ACR', 'MEANPOT', 'R_VALUE', 'SHRGT45', 'MEANSHR', 'MEANGAM',
       'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZH', 'MEANJZD', 'MEANALP', 'date_time'], how="left")

# Save the complete dataset into a csv file
complete_dataset.to_csv('all_the_data_we_need.csv', index=False)

<timed exec>:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_y'} in the result is deprecated and will raise a MergeError in a future version.


Wall time: 1min 49s


,Unnamed: 0_x,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,...,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0_y,X_or_M,flare_index,LOOK
0,0,2010.05.02_00:00:00_TAI,1,11067,11067,-65.205566,24.177683,298.638,18788750000000000327680.0,5949744000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2010.05.02_00:12:00_TAI,1,11067,11067,-64.984444,24.106541,301.313,19053479999999999737856.0,5977893000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2010.05.02_00:24:00_TAI,1,11067,11067,-64.88858,24.135088,312.556,19541660000000000655360.0,6152751000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2010.05.02_00:36:00_TAI,1,11067,11067,-64.758209,24.144226,316.364,21668099999999998296064.0,6234756000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2010.05.02_00:48:00_TAI,1,11067,11067,-64.534355,24.097631,331.849,24091639999999999213568.0,7389016000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900684,2900684,2021.12.30_23:12:00_TAI,7912,12923,12923,37.216583,-28.806856,17.246,1251446000000000065536.0,361816000000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900685,2900685,2021.12.30_23:24:00_TAI,7912,12923,12923,37.328621,-28.814804,19.899,1381736000000000000000.0,429238600000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900686,2900686,2021.12.30_23:36:00_TAI,7912,12923,12923,37.393547,-28.820169,23.266,1554694999999999901696.0,521120400000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900687,2900687,2021.12.30_23:48:00_TAI,7912,12923,12923,37.471878,-28.819061,24.068,1533790999999999901696.0,551538900000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
complete = pd.read_csv('all_the_data_we_need.csv', index_col=[0])
complete

C:\Users\joko\AppData\Local\Temp\ipykernel_32748\1584225701.py:1: DtypeWarning: Columns (27,28,30,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  complete = pd.read_csv('all_the_data_we_need.csv', index_col=[0])


,T_REC,HARPNUM,NOAA_ARS,NOAA_AR,LON_FWT,LAT_FWT,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,...,Unnamed: 0_y,Max,Type,Reg,CLASS,class,Unnamed: 0_y.1,X_or_M,flare_index,LOOK
Unnamed: 0_x,,,,,,,,,,,,,,,,,,,,,
0,2010.05.02_00:00:00_TAI,1,11067,11067,-65.205566,24.177683,298.638,1.878875e+22,5.949744e+12,17.197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010.05.02_00:12:00_TAI,1,11067,11067,-64.984444,24.106541,301.313,1.905348e+22,5.977893e+12,28.515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010.05.02_00:24:00_TAI,1,11067,11067,-64.888580,24.135088,312.556,1.954166e+22,6.152751e+12,27.740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010.05.02_00:36:00_TAI,1,11067,11067,-64.758209,24.144226,316.364,2.166810e+22,6.234756e+12,23.834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010.05.02_00:48:00_TAI,1,11067,11067,-64.534355,24.097631,331.849,2.409164e+22,7.389016e+12,14.075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900684,2021.12.30_23:12:00_TAI,7912,12923,12923,37.216583,-28.806856,17.246,1.251446e+21,3.618160e+11,0.845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900685,2021.12.30_23:24:00_TAI,7912,12923,12923,37.328621,-28.814804,19.899,1.381736e+21,4.292386e+11,0.018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2900686,2021.12.30_23:36:00_TAI,7912,12923,12923,37.393547,-28.820169,23.266,1.554695e+21,5.211204e+11,1.395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
